In [1]:
# !wget https://www.mediafire.com/file_premium/4ye5onawt91cm3n/curated_intervals.zip/file2

In [2]:
# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/file2'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:

# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/tesis-main.zip'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

In [5]:
#!wget https://www.mediafire.com/file_premium/etiia8akvx73qza/data.zip/

In [6]:
import sys
sys.path.append('/content/')

# from dotenv import load_dotenv

# load_dotenv()

In [7]:

# import drive as drive_downloader
# drive_downloader.download_files('14_UUqgoWDPPNwXvdFjDoJomHTu5l85l6')

* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [8]:
!pip install wandb

In [9]:
#!wget https://www.mediafire.com/file/4h0w0ulea622av9/siameseResnet18Cosine.pt

In [10]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from experiment_framework import experiment
from validation import VALIDATION_DATASET
from utils import load_dataset
warnings.filterwarnings('ignore')
import wandb

torch.manual_seed(47)
import random
random.seed(47)
import numpy as np
np.random.seed(47)

if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "mps"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)

dataset_loader = load_dataset(VALIDATION_DATASET.keys())

wandb.login()

wandb: Currently logged in as: piodanilod (hortensias). Use `wandb login --relogin` to force relogin


True

In [11]:
# !rm -r /content/data

In [12]:
!pip install ultralytics

In [13]:
from properties import MODELS_ENUM, PREPROCESSORS, ALGORITHM, GROUPER_FUNCTIONS

DEFAULT_PARAMS = {
      "dataset": dataset_loader,
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   }


AGGLOMERATIVE_PARAMS = {
            'distance_threshold':[0.5,1,2,4,5,7],
            'algorithm':["AGGLOMERATIVE"]
        }

AGGLOMERATIVE_TEMPLATE={
          'model': {
            'name': '{model_name}',
            'params': '{model_params}'
          },
          'algorithm': '{algorithm}',
          'params': { "n_clusters":None, "distance_threshold": "{distance_threshold}"},
          'grouper_function': "CLUSTERING_FUNCTION",
          'preprocessing': '{preprocessing}',
      }



SIMILARITY_PARAMS={
            "threshold":[.5,.8,.9,.99,.999,.9999,1,],
            "distance": ["euclidean","cosine"],
            'algorithm':["SIMILARITY"]
        }
SIMILARITY_TEMPLATE={
          'model': {
            'name': '{model_name}',
            'params': '{model_params}'
          },
          'algorithm': '{algorithm}',
          'params': { "threshold":'{threshold}', "distance":'{distance}' },
          'grouper_function': "CLUSTERING_FUNCTION",
          'preprocessing': '{preprocessing}',
      }


DBSCAN_PARAMS={
'algorithm':["DBSCAN"],
'eps' :[0.4,0.5,0.6,0.7],
'min_samples':[5,6,7,10,],
}
DBSCAN_TEMPLATE={
          'model': {
            'name': '{model_name}',
            'params': '{model_params}'
          },
          'algorithm': '{algorithm}',
          'params': {'eps' : '{eps}',
'min_samples':'{min_samples}'
},
          'grouper_function': "CLUSTERING_FUNCTION",
          'preprocessing': '{preprocessing}',
      }


In [14]:
from itertools import product
import copy
import json
import datetime


BASE_EXPERIMENTS_DEFINITIONS= [
    #  {
    #     'embeddings_params':{
    #         'preprocessing':["YOLO_BASELINE"],
    #         'model_name':['Encoder'],
    #         'model_params':[['best_rico.pt', 'autoencoderv5.pt', device]]
    #          #['best_rico.pt', 'autoencoderv5.pt', device]]
    #     },
    #   'name': "Encoder yolo mask clustering"
    # },




    #  {
    #     'embeddings_params':{
    #         'preprocessing':["BASELINE"],
    #         'model_name':['VGG19',"MOBILENETV2",'RESNET152','RESNET18'],
    #         'model_params':[[device]]

    #     },
    #   'name': "Pretrained Nets"
    # },

    #    {
    #     'embeddings_params':{
    #         'preprocessing':["YOLO_BASELINE"],
    #         'model_name':['YOLORES'],
    #         'model_params':[[device,'best_rico.pt'],[device,'best.pt'],[device,'yolov8n.pt']]

    #     },
    #   'name': "YOLO_RESNET"
    # },

    #   {
    #     'embeddings_params':{
    #         'preprocessing':["YOLO_BASELINE"],
    #         'model_name':["YOLOV8"],
    #         'model_params':[['best_rico.pt'],['best.pt'],['yolov8n.pt']]

    #     },
    #   'name': "YOLO"
    # },





     {
        'embeddings_params':{
            'preprocessing':["BASELINE"],
            'model_name':['SIAMESE_RESNET18'],
            'model_params':[[device,'siameseResnetCuratedEuclidea.pt' ]]


            # 'model_params':[[device,'siameseResnet18Cosine.pt' ],
            #                 [device,'siameseResnet18EuclideanFull.pt' ]]

        },
      'name': "Siamese Resnet"
    },


]
def get_experiment_definitions(exps):
  exps_emb = exps['embeddings_params']
  params  = {**exps_emb, **exps['other_params']}
  config = exps['template']
  name = exps['name']


  # Función recursiva para reemplazar valores en el JSON
  def replace_values(d, key, value_to_replace):
      if isinstance(d, dict):
          for k, v in d.items():
              if isinstance(v, dict) or isinstance(v, list):
                  replace_values(v, key, value_to_replace)
              elif isinstance(v, str) and v == "{" + key + "}":
                  d[k] = value_to_replace
      elif isinstance(d, list):
          for i, item in enumerate(d):
              if isinstance(item, dict) or isinstance(item, list):
                  replace_values(item, key, value_to_replace)
              elif isinstance(item, str) and item == "{" + key + "}":
                  d[i] = value_to_replace

  # Generar todas las combinaciones de los valores en params
  combinations = list(product(*params.values()))

  configs_modified = []

  for combination in combinations:
      # Crear una copia profunda del JSON original para cada combinación
      config_modified = copy.deepcopy(config)
      params_log =  zip(params.keys(), combination)
      name_components = [name]
      embeddings_hyperparams = []
      for key, value in params_log:
          replace_values(config_modified, key, value)
          name_components.append(f"{key}={value}")
          if key in exps_emb.keys():
              embeddings_hyperparams.append(f"{key}={value}")
      config_modified['raw_name'] = name
      config_modified['name'] = ', '.join(name_components)
      config_modified['hyperparams'] = ', '.join(embeddings_hyperparams)
      configs_modified.append(config_modified)

  # Imprimir los JSON modificados para verificar
  # for config in configs_modified:
  #     print(json.dumps(config, indent=4))
  return configs_modified


def add_clustering_method(exps):
    result = []
    for exp in exps:
        aglomerative = copy.deepcopy(exp)
        aglomerative['name'] = aglomerative['name']+" Agglomerative"
        aglomerative_dict = {
            'other_params':AGGLOMERATIVE_PARAMS,
            'template':AGGLOMERATIVE_TEMPLATE,
        }

        aglomerative={**aglomerative, **aglomerative_dict}


        simalarity = copy.deepcopy(exp)
        #simalarity['name'] = simalarity['name']+" Similarity"
        simalarity_dict = {
            'other_params':SIMILARITY_PARAMS,
            'template':SIMILARITY_TEMPLATE,
            'name': simalarity['name'] + " Similarity"
        }
        simalarity={**simalarity, **simalarity_dict}

        dbscan=copy.deepcopy(exp)
        dbscan['name'] = dbscan['name']+" DBSCAN"
        dbscan_dict = {
            'other_params':DBSCAN_PARAMS,
            'template':DBSCAN_TEMPLATE,
        }
        dbscan={**dbscan, **dbscan_dict}
        result.append(aglomerative)
        result.append(simalarity)
        result.append(dbscan)
    return result


EXPERIMENTS_DEFINITIONS = add_clustering_method(BASE_EXPERIMENTS_DEFINITIONS)
EXPERIMENT_DEFINITIONS = []



for exps_definition in EXPERIMENTS_DEFINITIONS:
  EXPERIMENT_DEFINITIONS.extend(    get_experiment_definitions(exps_definition))


number_of_experiments = len(EXPERIMENT_DEFINITIONS)


i=0
start = datetime.datetime.now()
for exp_definition in EXPERIMENT_DEFINITIONS:
    i=i+1
    if i>0:
      params = DEFAULT_PARAMS.copy()
      model_definition = MODELS_ENUM[exp_definition["model"]["name"]]
      model_params = exp_definition["model"]["params"]
      model = model_definition(*model_params)
      params["model"] = model
      params["preprocessing"] = PREPROCESSORS[exp_definition["preprocessing"]]
      algorithm = ALGORITHM[exp_definition["algorithm"]](**exp_definition["params"])
      grouper_function = GROUPER_FUNCTIONS[exp_definition["grouper_function"]](algorithm)
      params["grouper_function"] = grouper_function
      params["name"] = exp_definition["name"]
      params["dataset"] = dataset_loader()
      params['hyperparams']=exp_definition["hyperparams"]
      result = experiment(**params)
      run = wandb.init(project="run-all-dataset-curado",group=exp_definition['raw_name'], entity="hortensias")
      run.log({
        "PCA_FILE":model.get_short_name(params['hyperparams']),
        "hyperparams": exp_definition['name'],
        "Precision": result.precision.mean,
        "Accuracy": result.accuracy.mean,
        "F1": result.f1.mean,
        "Recall": result.recall.mean,
        "Precision_STD": result.precision.std,
        "Accuracy_STD": result.accuracy.std,
        "F1_STD": result.f1.std,
        "Recall_STD": result.recall.std,
      })
      run.finish()

    avance = i / number_of_experiments
    diferencia = datetime.datetime.now() - start
    tiempo_restante = ((1 - avance) / avance) * diferencia.total_seconds()
    tiempo_restante = datetime.timedelta(seconds=tiempo_restante)

    horas = tiempo_restante.seconds // 3600
    minutos = (tiempo_restante.seconds % 3600) // 60
    segundos = tiempo_restante.seconds % 60

    print(f"Se han corrido {i} de {number_of_experiments} experimentos. {avance * 100:.2f} % avance, tiempo restante: {horas}h{minutos:02d}m{segundos:02d}s")


Experiment name: Siamese Resnet Agglomerative, preprocessing=BASELINE, model_name=SIAMESE_RESNET18, model_params=[device(type='cuda', index=0), 'siameseResnetCuratedEuclidea.pt'], distance_threshold=0.5, algorithm=AGGLOMERATIVE AVG
Precision - Mean: 0.94, std: 0.0099327248
Recall - Mean: 0.23, std: 0.0229673545
F1-Score - Mean: 0.34, std: 0.0198906058
Accuracy - Mean: 0.23, std: 0.0229673545


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.23281
Accuracy_STD,0.02297
F1,0.3351


Se han corrido 1 de 36 experimentos. 2.78 % avance, tiempo restante: 0h05m17s

Experiment name: Siamese Resnet Agglomerative, preprocessing=BASELINE, model_name=SIAMESE_RESNET18, model_params=[device(type='cuda', index=0), 'siameseResnetCuratedEuclidea.pt'], distance_threshold=1, algorithm=AGGLOMERATIVE AVG
Precision - Mean: 0.94, std: 0.0108453331
Recall - Mean: 0.28, std: 0.0407595247
F1-Score - Mean: 0.39, std: 0.0392270181
Accuracy - Mean: 0.28, std: 0.0407595247


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.2837
Accuracy_STD,0.04076
F1,0.38788


Se han corrido 2 de 36 experimentos. 5.56 % avance, tiempo restante: 0h04m32s

Experiment name: Siamese Resnet Agglomerative, preprocessing=BASELINE, model_name=SIAMESE_RESNET18, model_params=[device(type='cuda', index=0), 'siameseResnetCuratedEuclidea.pt'], distance_threshold=2, algorithm=AGGLOMERATIVE AVG
Precision - Mean: 0.89, std: 0.0127833874
Recall - Mean: 0.29, std: 0.0068979833
F1-Score - Mean: 0.40, std: 0.0048174061
Accuracy - Mean: 0.29, std: 0.0068979833


...

Se han corrido 34 de 36 experimentos. 94.44 % avance, tiempo restante: 0h00m17s

Experiment name: Siamese Resnet DBSCAN, preprocessing=BASELINE, model_name=SIAMESE_RESNET18, model_params=[device(type='cuda', index=0), 'siameseResnetCuratedEuclidea.pt'], algorithm=DBSCAN, eps=0.7, min_samples=7 AVG
Precision - Mean: 0.52, std: 0.0333705199
Recall - Mean: 0.30, std: 0.1703113538
F1-Score - Mean: 0.28, std: 0.1349662232
Accuracy - Mean: 0.30, std: 0.1703113538


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.30346
Accuracy_STD,0.17031
F1,0.28195


Se han corrido 35 de 36 experimentos. 97.22 % avance, tiempo restante: 0h00m08s

Experiment name: Siamese Resnet DBSCAN, preprocessing=BASELINE, model_name=SIAMESE_RESNET18, model_params=[device(type='cuda', index=0), 'siameseResnetCuratedEuclidea.pt'], algorithm=DBSCAN, eps=0.7, min_samples=10 AVG
Precision - Mean: 0.42, std: 0.0613237890
Recall - Mean: 0.45, std: 0.0056596154
F1-Score - Mean: 0.38, std: 0.0101508080
Accuracy - Mean: 0.45, std: 0.0056596154


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.44634
Accuracy_STD,0.00566
F1,0.38161


Se han corrido 36 de 36 experimentos. 100.00 % avance, tiempo restante: 0h00m00s


In [15]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#VALIDATION_DATASET={"Alibaba.com.mp4": [[1, 31, "NONE", ""], [32, 149, "Home", ""], [150, 202, "Lists of items", ""], [203, 789, "Product details", ""], [790, 906, "Product item", ""], [907, 1043, "Product details", ""], [1044, 1149, "Cart screen confirmation", ""], [1150, 1200, "Cart Screen", ""], [1201, 1225, "Product information", ""], [1226, 1292, "List of items", ""], [1293, 1374, "Home", ""], [1375, 1427, "Cart screen", ""]], "chrome_post_post.mp4": [[1, 34, "Home", ""], [35, 131, "search menu", ""], [132, 550, "list of posts", ""], [551, 573, "Cookies settings", ""], [574, 789, "Post information", ""], [790, 1003, "List of Posts", ""], [1004, 1238, "Post 2", ""], [1239, 1239, "NONE", ""]]}

In [21]:
import shutil

# Define el nombre del archivo ZIP que deseas crear (sin la extensión .zip)
archivo_zip = 'embeddings_8'

# Define la carpeta que quieres comprimir
carpeta_a_comprimir = '/content/embeddings'

# Crea el archivo ZIP
shutil.make_archive(archivo_zip, 'zip', carpeta_a_comprimir)

'/content/embeddings_8.zip'

In [22]:
!cp embeddings_8.zip /content/drive/MyDrive/embeddings_8.zip

In [19]:
from google.colab import files

# Reemplaza 'nombre_de_tu_archivo.ext' con el nombre real de tu archivo
files.download('/content/embeddings_8.zip')